In [86]:
#imports, global consts, inits
import ee
import geemap

NDVI_VIZ_PARAMS = {"min" : -1, "max": 1, "palette" : ["blue", "white", "green"]}
NDVI_VIZ_PARAMS_CROP_ONLY = {"min" : 0.1, "max": 0.5, "palette" : ["red", "yellow", "green"]}


ee.Initialize(project='seamproj01')
map = geemap.Map()

sudanStateBorders = ee.FeatureCollection("projects/seamproj01/assets/SudanStateBorders") #Shapefiles for Sudan administration borders, via OCHA HDX
#sudanCroplandMask = ee.FeatureCollection("projects/seamproj01/assets/Sudan_Cropland_Mask_CopernicusLCLU2019") #shapefile generated from Copernicus Moderate Dynamic Land Cover dataset.
testArea =  ee.FeatureCollection("projects/seamproj01/assets/test_area").geometry()

khartoum = sudanStateBorders.filter(ee.Filter.inList("ADM1_EN", rightValue=["Khartoum"])).geometry()
gezira = sudanStateBorders.filter(ee.Filter.inList("ADM1_EN", rightValue=["Aj Jazirah"])).geometry()

# khartoumCropland = sudanCroplandMask.filter(ee.Filter.inList("ADM1_EN", rightValue=["Khartoum"])).geometry()
# geziraCropland = sudanCroplandMask.filter(ee.Filter.inList("ADM1_EN", rightValue=["Aj Jazirah"])).geometry()

khartoumCroplandModified = ee.FeatureCollection("projects/seamproj01/assets/khartoum_cropmask_gt_1km2").geometry()
geziraCroplandModified = ee.FeatureCollection("projects/seamproj01/assets/gezira_cropmask_western_bank").geometry()

In [87]:
def AddNDVI(col : ee.ImageCollection) -> ee.ImageCollection:
    def NDVIize(img : ee.Image) -> ee.Image:
        return img.addBands(img.normalizedDifference(["sur_refl_b02", "sur_refl_b01"]).rename("NDVI"))
    return col.map(NDVIize)


def GetQualityMask(img : ee.Image) -> ee.Image:
    def bitwiseExtract(value, fromBit, toBit = None):
    #https://gis.stackexchange.com/questions/349371/creating-cloud-free-images-out-of-a-mod09a1-modis-image-in-gee
        if (toBit == None):
             toBit = fromBit
        maskSize = ee.Number(1).add(toBit).subtract(fromBit)
        mask = ee.Number(1).leftShift(maskSize).subtract(1)
        return value.rightShift(fromBit).bitwiseAnd(mask)
    
    state = img.select("State")

    stateCloud = bitwiseExtract(state, 0, 1)
    stateCloudShadow = bitwiseExtract(state, 2)
    stateLC = bitwiseExtract(state, 3, 5)
    stateCirrus = bitwiseExtract(state, 8, 9)
    
    mask = stateCloud.eq(0).Or(stateCloud.eq(3)) #clear pixel or unset cloud state (assumed clear)
    mask = mask.And(stateCloudShadow.eq(0)) #no cloud shadow
    mask = mask.And(stateLC.eq(1)) #land
    mask = mask.And(stateCirrus.eq(0).Or(stateCirrus.eq(1))) #no or low cirrus clouds (TODO redundant? check modis docs)
    return mask


def MaskOutPoorQualityPixels(col : ee.ImageCollection) -> ee.ImageCollection:
    def UpdateMask(img : ee.Image) -> ee.Image:
        mask = GetQualityMask(img)
        return img.updateMask(mask)
    
    return col.map(UpdateMask)

In [88]:
#TODO "roi" is redundant. Remove it and replace any uses bellow with roiCropMask
roi = gezira
roiCropMask = geziraCroplandModified

#time-series limits. All inclusive.
yearStart = 2017
monthStart = 1
yearEnd = 2024
monthEnd = 6

#targetMonths are the months comprising the season for analysis.
#WARNING! MUST BE 4 Values!
    #Otherwise, the Temporal Anomaly Analysis component must be adjusted
#WARNING! ORDER OF MONTHS MUST BE CHRONOLOGICALLY! If the season is inter-annual, start with the months in the first year, then the second year
    #e.g. if the season starts on November, the list would be [11, 12, 1, 2]
#targetMonths = [7, 8, 9, 10] #"Summer" season in Sudan (technically Autumn). This covers the growth periods of crops such as sorghum.
targetMonths = [11, 12, 1, 2] #"Winter" season in Sudan. This covers growth periods of crops such as wheat

#Grabbing the MODIS dataset, filtering for time and roi, masking out poor quality pixels and those outside the cropland, and computing the NDVI.
dateStart = f"{yearStart}-{monthStart}-1"
dateEnd = f"{yearEnd}-{monthEnd + 1}-1" if monthEnd < 12 else  f"{yearEnd + 1}-1-1"

modis = ee.ImageCollection("MODIS/061/MOD09Q1")
modis = modis.filterBounds(roi).filterDate(dateStart, dateEnd)
modis = AddNDVI(MaskOutPoorQualityPixels(modis)).map(lambda img : img.select("NDVI").clip(roiCropMask))


In [89]:
#generting the timeseries (monthly max NDVI and NDVI range) and the statistics for the "pure crop" signal.
#Outputs of this block are two ImageCollections: "timeseries," and "pureCropNDVI"
    #timseries contains images for each season.
        #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
        #Each Image has a property "year" for the season's year, and property "isComplete" showing whether available data covers all targetMonths (1 = true, 0 = false)
        #Each band is named "month_x_max" or "month_x_range", where x is the month's number; "max" and "range" denote whether it encodes the maximum monthly ndvi or ndvi monthly range.
    #pureCropNDVI contains images for each month in the targetMonths (total = 4)
        #each image has 4 bands: "month_x_max_mean" or "month_x_max_stdDev" (and similarily for range).
        #each image has a property "month" with month's number

timeSeries = ee.List([])

for year in range(yearStart, yearEnd + 1):
    yearTS = ee.List([])
    isCompleteYear = 1
    #for month in range(1, 13):
    for month in targetMonths:
        if ((year * 100) + month > (yearEnd * 100) + monthEnd):
            #add a fake (empty) band, else GEE would throw a fit for selection of a non-existence band.
            fakeBand = ee.Image([ee.Image(), ee.Image()]).rename(["max", "range"]).set({"system:index" : f"month_{month}"})
            yearTS = yearTS.add(fakeBand)
            isCompleteYear = 0
            continue

        subPeriodStart = f"{year}-{month}-1"
        subPeriodEnd = f"{year}-{month + 1}-1" if month < 12 else  f"{year + 1}-1-1"
        subCol = modis.filterDate(subPeriodStart, subPeriodEnd)
        
        minMaxNDVI = subCol.reduce(ee.Reducer.minMax())

        monthMax = minMaxNDVI.select("NDVI_max").rename("max")
        monthRange = minMaxNDVI.select("NDVI_max").subtract(minMaxNDVI.select("NDVI_min")).rename("range")
        yearTS = yearTS.add(ee.Image([monthMax, monthRange]).set({"system:index" : f"month_{month}"}))
    
    yearTS = ee.ImageCollection(yearTS).toBands().set({"year" : year, "isComplete" : isCompleteYear})
    timeSeries = timeSeries.add(yearTS)

timeSeries = ee.ImageCollection(timeSeries)

#TODO I made the monthlyMedianNDVI imagecollection in an attempt to do some testing of the internal workings of the algorithm. It exactly usefull for any other purpose (I can think of atm). Consider removing.
monthlyMedianNDVI = ee.List([])
pureCropNDVI = ee.List([])

#for month in range(1, 13):
for month in targetMonths:
    prefix = f"month_{month}_"
    thisMonthTS = timeSeries.select([prefix + "max", prefix + "range"])

    monthMedian = thisMonthTS.reduce(ee.Reducer.median()).rename([prefix + "max", prefix + "range"]).set({"month" : month})
    
    monthlyMedianNDVI = monthlyMedianNDVI.add(monthMedian)
    
    meanStdReducer = ee.Reducer.mean().combine(ee.Reducer.stdDev(), sharedInputs = True)

    monthPureCropNDVI = thisMonthTS.map(lambda img : img.updateMask(img.gt(monthMedian)))
    monthPureCropNDVI = monthPureCropNDVI.reduce(meanStdReducer).set("month", month)

    pureCropNDVI = pureCropNDVI.add(monthPureCropNDVI)

monthlyMedianNDVI = ee.ImageCollection(monthlyMedianNDVI)
pureCropNDVI = ee.ImageCollection(pureCropNDVI)

In [90]:
#This block computes the temporal anomalies for each season. The output is an ImageCollection called temporalAnomalies, containing Images representing each season.
    #Each Image has number of bands equal to twice the number of targetMonths (2x4 = 8).
    #Each Image has a property "year" for the season's year.
    #Each band is named "month_x_max" or "month_x_range".
#note: The loop handles inter-annual years (assuming the targetMonth list was set correctly). However, the output of the anomalies will be given a "year" property based on its season, not its
#calendar year. For example, for a season that starts on Nov 2013 to Feb 2014, the months of Jan and Feb would be bands in the image with "year" = 2013.

temporalAnomalies = ee.List([])

for year in range (yearStart, yearEnd + 1):
    seasonAnomalies = ee.List([])

    #for month in range(1, 13):
    lastMonth = 0
    calendarYear = year
    incompleteYear = False
    
    for month in targetMonths:       
        if (month < lastMonth):
            calendarYear += 1
        lastMonth = month

        if ((calendarYear * 100) + month > (yearEnd * 100) + monthEnd):
            incompleteYear = True
            break

        prefix = f"month_{month}_"
        pureCropSignal = pureCropNDVI.filter(ee.Filter.eq("month", month)).first()
        
        ta = timeSeries.filter(ee.Filter.eq("year", calendarYear)).first()

        taMax = ta.select(prefix + "max").subtract(pureCropSignal.select(prefix + "max_mean")).divide(pureCropSignal.select(prefix + "max_stdDev")).rename("max")
        taRange = ta.select(prefix + "range").subtract(pureCropSignal.select(prefix + "range_mean")).divide(pureCropSignal.select(prefix + "range_stdDev")).rename("range")

        ta = ee.Image([taMax, taRange]).set({"system:index" : f"month_{month}"})
        seasonAnomalies = seasonAnomalies.add(ta)

    if (not incompleteYear):
        seasonAnomalies = ee.ImageCollection(seasonAnomalies).toBands().set({"year" : year})
        temporalAnomalies = temporalAnomalies.add(seasonAnomalies)

temporalAnomalies = ee.ImageCollection(temporalAnomalies)

In [91]:
#Temporal analysis component

def TemporalAnomalyAnalysis(image : ee.Image) -> ee.Image: #to be mapped over temporalAnomalies collection
    taMax = []
    taRange = []

    for month in targetMonths:
        name = f"month_{month}"
        taMax.append(image.select(name + "_max").rename(name))
        taRange.append(image.select(name + "_range").rename(name))
        
    isFallow_1 = taMax[0].lt(-3).And(taMax[1].lt(-3)).And(taMax[2].lt(-3))
    isFallow_1 = isFallow_1.Or(taMax[1].lt(-3).And(taMax[2].lt(-3)).And(taMax[3].lt(-3)))
    isFallow_1 = isFallow_1.rename("max")

    isFallow_2 = taRange[0].lt(-3).And(taRange[1].lt(-3)).And(taRange[2].lt(-3))
    isFallow_2 = isFallow_2.Or(taRange[1].lt(-3).And(taRange[2].lt(-3)).And(taRange[3].lt(-3)))
    isFallow_2 = isFallow_2.rename("range")

    return ee.Image([isFallow_1, isFallow_2]).set({"year" : image.get("year")})

#isFallow_TA the two questions for the temporal anomalies
isFallow_TA = temporalAnomalies.map(TemporalAnomalyAnalysis)

In [92]:
def SpatialAnomalyAnalysis(image : ee.Image) -> ee.Image:
    #image has max and range for each month in a year (12 bands). month_x_max, month_x_range, x = month number
    maxOfMax = ee.List([])
    maxOfRange = ee.List([])
    
    spatialMedianMax = ee.List([])
    spatialMedianRange = ee.List([])

    for month in targetMonths:
        name = f"month_{month}"
        _maxOfMax = image.select(name + "_max").rename("max")
        _maxOfRange = image.select(name + "_range").rename("range")
        
        maxOfMax = maxOfMax.add(_maxOfMax)
        maxOfRange = maxOfRange.add(_maxOfRange)


        spatialMedianMax =      spatialMedianMax.add(   _maxOfMax.reduceRegion(
                                                        reducer = ee.Reducer.median(),
                                                        geometry = roiCropMask,
                                                        tileScale = 4,
                                                        scale = 250,
                                                        crs='EPSG:4326',
                                                        ).getNumber("max"))
        
        spatialMedianRange =   spatialMedianRange.add(  _maxOfRange.reduceRegion(
                                                        reducer = ee.Reducer.median(),
                                                        geometry = roiCropMask,
                                                        tileScale = 4,
                                                        scale = 250,
                                                        crs='EPSG:4326',
                                                        ).getNumber("range"))


    maxOfMax = ee.ImageCollection(maxOfMax)
    maxOfRange = ee.ImageCollection(maxOfRange)

    spatialMedianMax = ee.Number(spatialMedianMax.reduce(ee.Reducer.max()))
    spatialMedianRange = ee.Number(spatialMedianRange.reduce(ee.Reducer.max()))

    isFallow_3 = maxOfMax.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianMax)).set({"year" : year}).rename("max")
    isFallow_4 = maxOfRange.reduce(ee.Reducer.max()).lt(ee.Number(0.8).multiply(spatialMedianRange)).set({"year" : year}).rename("range")

    return ee.Image([isFallow_3, isFallow_4]).set({"year" : image.get("year")})

#isFallow_SA the two questions for the spatial anomalies
isFallow_SA = timeSeries.filter(ee.Filter.eq("isComplete", 1)).map(SpatialAnomalyAnalysis)

In [93]:
#Final analysis

isFallowComponents = ee.List([])
for year in range (yearStart, yearEnd + 1):
    if ((year * 100) + targetMonths[0] > (yearEnd * 100) + monthEnd):
            break
    
    yearComponents = ee.List([]).add(isFallow_TA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_TA"}))
    yearComponents = yearComponents.add(isFallow_SA.filter(ee.Filter.eq("year", year)).first().set({"system:index" : f"{year}_SA"}))
    isFallowComponents = isFallowComponents.add(ee.ImageCollection(yearComponents).toBands().set({"year" : year}))

isFallowComponents = ee.ImageCollection(isFallowComponents)

fallowTS = isFallowComponents.map(lambda img : img.reduce(ee.Reducer.sum()).gte(ee.Number(2)).rename(ee.String(img.get("year")))).toBands()

In [94]:
#export

noDataValue = -9999

task = ee.batch.Export.image.toDrive(
    image=fallowTS,
    description= f"fallow_ts_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[3]}",
    #folder='ee_export',
    region=roi,
    scale=250,
    crs='EPSG:4326',
    fileFormat='GeoTIFF',
    formatOptions={
        'noData': noDataValue
    })

task.start()

# isFallowComponentsCollapsed = isFallowComponents.toBands()

# task = ee.batch.Export.image.toDrive(
# image=isFallowComponentsCollapsed,
# description=f"fallow_components_{yearStart}-{yearEnd}_season_{targetMonths[0]}-{targetMonths[3]}",
# #folder='ee_export',
# region=roi,
# scale=250,
# crs='EPSG:4326',
# fileFormat='GeoTIFF',
# formatOptions={
#     'noData': noDataValue
# })

#task.start()

In [95]:
# map.centerObject(roi)
# map.addLayer(fallowTS, name="FallowTS")
# map